# How to use
Since the segmented textures are hard coded, this script is designed to be used to produce both the images and the segmented images. To determine what kind of images to generate, set the boolean value below and swap in the right `texture` and `meshes` folders.

Use `generate_valid_positions.ipynb` to generate the valid positions first before running this script

In [1]:
import sys
import argparse
import pyglet
from pyglet.window import key
import numpy as np
import gym
import gym_duckietown
from gym_duckietown.envs import DuckietownEnv
from gym_duckietown.wrappers import UndistortWrapper

In [3]:
segmented = True

### Read valid positions file

In [4]:
import pickle
filepath = "segmentation/valid_positions.pkl"
f = open(filepath, "rb")
valid_positions = pickle.load(f)
f.close()

current_map = valid_positions['current_map']
size_n = len(valid_positions['angle'])

print("Current map: ", current_map)
print("Size of data: ", size_n)

Current map:  udem1
Size of data:  200


## Generate segmented images

Need to convert RGB image into label IDs. `labels_info` describe the color to id mapping for the objects that we are concerned with.

In [5]:
# Used to allow training of bisenet
labels_info = [
    { "name": "background", "id": 0, "color": [0, 0, 0], "trainId": 0 },         # black
    { "name": "sky", "id": 1, "color": [0, 0, 255], "trainId": 1 },              # blue
    { "name": "road", "id": 2, "color": [0, 255, 0], "trainId": 2 },             # green
    { "name": "midlane", "id": 3, "color": [255, 255, 0], "trainId": 3 },        # yellow
    { "name": "road_border", "id": 4, "color": [255, 255, 255], "trainId": 4 },  # white
]

# Used to force ill-defined colors to belong to one of the colors in this list
palette = [label['color'] for label in labels_info]
palette = np.array(palette)

# Generate RGB lookup table before hand
color_map = np.zeros(shape=(256*256*256), dtype='int32')
for label in labels_info:
    color = label['color']
    rgb = color[0] * 65536 + color[1] * 256 + color[2]
    color_map[rgb] = label['id']

def rgb_to_label(image, color_map):
    image = image.dot(np.array([65536, 256, 1], dtype='int32'))
    return color_map[image]

Initialize environment

In [6]:
env = DuckietownEnv(
    map_name = current_map,
    domain_rand = False # To ensure camera settings are the same across all images
)
obs = env.reset()

In [7]:
from PIL import Image
from scipy.spatial import cKDTree

identifier = "img"

for i in range(size_n):
    # Set position
    pos, angle = valid_positions['pos'][i], valid_positions['angle'][i]
    env.set_agent_info(pos, angle, verbose=False)
    img = env.render('rgb_array')
    
    if segmented:
        identifier = "label"
        
        # Force ill-defined colors to colors in palette
        img = palette[cKDTree(palette).query(img,k=1)[1]]

        # Convert colors to label
        img = rgb_to_label(img, color_map)
        img = img.astype(np.uint8)
        
        # Sanity check
        if i == 0:
            # Map labels to more obvious grayscale colors
            lbmap = np.array([0, 200, 255, 150, 200])
            check = lbmap[img]
            check = check.astype(np.uint8)
            check = Image.fromarray(check)
            check.show()
    

    # Resize image to handle Bisenet model i.e. width and height must be divisible by 2^5
    # Pad height by 8
    if segmented:
        img = np.pad(img, ((4,4), (0,0)), mode="edge")
    else:
        img = np.pad(img, ((4,4), (0,0), (0,0)), mode="edge")

    # Save image
    im = Image.fromarray(img)
    im.save(f"segmentation/{identifier}{i}.png")

env.close()
print(f"{size_n} images saved!")

200 images saved!


## Generate annotation file

In [4]:
with open("segmentation/annotation.txt", 'w') as f:
    for i in range(size_n):
        f.write(f"img{i}.png,label{i}.png\n")